In [1]:
%load_ext cython

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%%cython

from cpython.pycapsule cimport PyCapsule_New

import cython
from libc.math cimport pi, sqrt, exp
from scipy.special.cython_special cimport wofz, exp1

cdef extern from "complex.h":
    double creal(double complex)
    double cimag(double complex)


@cython.cdivision(True)
cdef double c_kernel(int n, double[2] args):

    cdef double x, q, xi, W_r, W_i, D_r, D_i
    cdef double complex w

    x = args[0]
    q = args[1]

    xi = sqrt(0.5)*x

    w = wofz(xi)

    W_r = 1 - sqrt(pi)*xi*cimag(w)
    W_i = sqrt(pi)*xi*creal(w)

    D_r = 1 - q*q*W_r
    D_i =   - q*q*W_i

    return exp(-xi*xi)*exp1(xi*xi)/(D_r*D_r + D_i*D_i)


def kernel(x, q):

    cdef double[2] args

    args[0] = x
    args[1] = q

    return c_kernel(2, args)


def get_LowLevelCallable():

    from scipy import LowLevelCallable

    func_capsule = PyCapsule_New(<void*>c_kernel, "double (int, double *)", NULL)

    return LowLevelCallable(func_capsule)


Error compiling Cython file:
------------------------------------------------------------
...

from cpython.pycapsule cimport PyCapsule_New

import cython
from libc.math cimport pi, sqrt, exp
from scipy.special.cython_special cimport wofz, exp1
^
------------------------------------------------------------

/Users/tobson/.cache/ipython/cython/_cython_magic_77f3a342ab51d62b0671202721477a19.pyx:6:0: 'scipy/special/cython_special.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...

from cpython.pycapsule cimport PyCapsule_New

import cython
from libc.math cimport pi, sqrt, exp
from scipy.special.cython_special cimport wofz, exp1
^
------------------------------------------------------------

/Users/tobson/.cache/ipython/cython/_cython_magic_77f3a342ab51d62b0671202721477a19.pyx:6:0: 'scipy/special/cython_special/wofz.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...

from 

In [4]:
from math import pi, sqrt, exp
from scipy.special import wofz, exp1

def kernel2(x, q):
    ξ = sqrt(0.5)*x
    W = 1 + 1j*sqrt(pi)*ξ*wofz(ξ)
    D = 1 - q*q*W
    return exp1(ξ*ξ)*exp(-ξ*ξ)/abs(D)**2

for i in range(10):
    x = np.random.randn()
    q = np.random.rand()
    assert np.isclose(kernel(x, q), kernel2(x, q))

ModuleNotFoundError: No module named 'scipy'

In [ ]:
from scipy.integrate import quad

def integrate_and_sum(q0, ninv, lower=-np.inf, upper=np.inf):
    
    func = get_LowLevelCallable()
    
    G_bare = np.zeros_like(q0)
    G_dressed = np.zeros_like(q0)
    max_error = 0

    for j in range(q0.size):

        for n1 in ninv:

            q = q0[j]*n1
            q3 = q**3

            G_bare[j] += q3

            dressed, error = quad(func, lower, upper, args=(q,))
            max_error = max(error, max_error)
            G_dressed[j] += q3*dressed
            
        bare, error = quad(func, lower, upper, args=(0,))
        max_error = max(error, max_error)
        G_bare[j] *= bare
        
        print("\r{:.0%}".format((j+1)/q0.size), end='', flush=True)
        
    print("\nMaximum error: {}".format(max_error))
    
    return G_bare, G_dressed

In [ ]:
nmax = 5
n2 = []
for nx in range(-nmax, nmax + 1):
    for ny in range(-nmax, nmax + 1):
        for nz in range(-nmax, nmax + 1):
            n2.append(nx*nx + ny*ny + nz*nz)
n2 = np.array(n2)
mask = np.logical_or(n2 == 0, n2 > nmax*nmax)
n2_compressed = np.ma.masked_where(mask, n2).compressed()
n2_sorted = np.sort(n2_compressed)
ninv = 1/np.sqrt(n2_sorted)

In [ ]:
q0 = np.linspace(0.01, 0.99, 128)
G_bare, G_dressed = integrate_and_sum(q0, ninv, lower=0)

plt.semilogy(q0, G_dressed/G_bare - 1);